## Introdução ao Aprendizado de Máquina
Hoje vamos aprender alguns conceitos básicos sobre aprendizado de máquina como pré-processamento e validação de experimentos.

In [2]:
! git clone https://github.com/SrtaCamelo/DataMining_2019.1

Cloning into 'DataMining_2019.1'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 21 (delta 5), reused 21 (delta 5), pack-reused 0
Unpacking objects: 100% (21/21), done.


## Abrindo e Pré- Processando o Data-set

### Abrindo em TXT O dataSet Iris
Link do UCI
https://archive.ics.uci.edu/ml/datasets/iris

In [0]:
path = "DataMining_2019.1/Aula/DataSets/original/iris_dataSet.txt"
f = open(path,"r")
text = f.read()
print(text)

### Transformando o arquivo TXT em CSV

In [0]:
import csv

def txt_2_csv(infile,outfile, header):
    with open(outfile,'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(header)
    with open(infile) as infile, open(outfile, 'a') as outfile:
        for line in infile:
            outfile.write(line.replace(' ', ','))
            

filename = "iris_csv.csv"
header = header = ['sepal length', 'sepal width','petal length', 'petal width','class']
txt_2_csv(path,filename,header)

### Abrindo o novo CSV para realiza ro pré-processamento com o Pandas

Link da biblioteca: https://pandas.pydata.org/

Link de tutorial: https://medium.com/data-hackers/uma-introdu%C3%A7%C3%A3o-simples-ao-pandas-1e15eea37fa1

Pandas: https://thispointer.com/select-rows-columns-by-name-or-index-in-dataframe-using-loc-iloc-python-pandas/

In [36]:
import pandas as pd
data = pd.read_csv(filename)
print(data)

     sepal length  sepal width  petal length  petal width           class
0             5.1          3.5           1.4          0.2     Iris-setosa
1             4.9          3.0           1.4          0.2     Iris-setosa
2             4.7          3.2           1.3          0.2     Iris-setosa
3             4.6          3.1           1.5          0.2     Iris-setosa
4             5.0          3.6           1.4          0.2     Iris-setosa
5             5.4          3.9           1.7          0.4     Iris-setosa
6             4.6          3.4           1.4          0.3     Iris-setosa
7             5.0          3.4           1.5          0.2     Iris-setosa
8             4.4          2.9           1.4          0.2     Iris-setosa
9             4.9          3.1           1.5          0.1     Iris-setosa
10            5.4          3.7           1.5          0.2     Iris-setosa
11            4.8          3.4           1.6          0.2     Iris-setosa
12            4.8          3.0        

### Pré- processando os dados
A normalização dos dados é necessária para garantir que todos os dados estão em um padrão (escala). Isso é feito por dois motivos principais: Fazer com que so classificadores enxerguem todos os dados de uma maneira justa (nenhum é mais importante que o outro) 
A maioria dos algoritmos de aprendizado de máquina assumem que os dados estão padronizados quando vão gerar seus modelos.

Link útil: https://minerandodados.com.br/index.php/2017/12/28/pre-processamento-standartization/

A discretização se trata em converter atributos nominais para números, geralmente dados qualitativos ou atributos de classe. 

Link útil: https://medium.com/brasil-ai/antes-de-come%C3%A7armos-a-falar-sobre-tipos-de-aprendizados-que-veremos-no-pr%C3%B3ximo-artigo-%C3%A9-ea5b04685913

Scikit Learn Preprocessing: https://scikitlearn.org/stable/modules/preprocessing.html

In [0]:
from sklearn import preprocessing

#--------------Change Labels to numbers---------
def numberize(column,tuple):
    new_column = []
    for item in column[1:]:
        for name in tuple:
            if item == name[1]:
                new_column.append(name[0])
    return new_column
#----------------Discretie Atributes (numbered)----------------
def discretizer2(data, column_to_disc,class_tuple):
    new_column = numberize(data[column_to_disc],class_tuple)
    new_column = pd.DataFrame({'clas':new_column})
    data = data.merge(new_column, left_index=True, right_index=True)
    del data[column_to_disc]
    return data

#-------------------Normalize 0 -1 range---------------
def normalizer(data,atributes, final_column):
    #normalized_df = (data - data.min()) / (data.max() - data.min())
    for atribute in atributes:
      
        x = data[[atribute]].values.astype(float)
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        df_normalized = pd.DataFrame(x_scaled)
        #print(df_normalized)
        data = data.merge(df_normalized,left_index=True, right_index=True)
        del data[atribute]
    data.columns = final_column
    return data
    #print(dt_amp)

#---------------List to Tuple-------------------
def tuplenizer(list):
    outlist = []
    for i in range(len(list)):
        tuple = (i,list[i])
        outlist.append(tuple)
    return outlist
#------------------Save Data Back into csv------------

def saveCSV(data,filename):
    data.to_csv(filename, header= True, mode='w',sep=',')
    
    
atributes_to_normalize = ['sepal length', 'sepal width','petal length', 'petal width']
final_column = ['class','sepal length','sepal width','petal length','petal width']
classes = ['Iris-setosa','Iris-versicolor','Iris-virginica']

filename_pdc = "iris_csv_preprocessado.csv"

class_tuple = tuplenizer(classes)


data = normalizer(data,atributes_to_normalize,final_column)
data = discretizer2(data,'class',class_tuple)
print(data)

saveCSV(data,filename_pdc)

## Escolhendo O classificador

https://scikit-learn.org/stable/supervised_learning.html#supervised-learning

In [0]:
#----------------Imports--------------------
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from scipy import stats

#--------------Structures for Score Validation----------
tree_acuracy = []
nb_acuracy = []
knn_1_acuracy = []
wknn_1_acuracy = []
knn_3_acuracy = []
wknn_3_acuracy = []
knn_5_acuracy = []
wknn_5_acuracy = []
svm_linear_acuracy = []
svm_rbf_acuracy = []


def call_classifiers(x_train, y_train, x_test, y_test):
#---------------Training Models-------------
    #----------------Decision Tree----------

    tree = DecisionTreeClassifier(criterion = "entropy", random_state = 100,
    max_depth=3, min_samples_leaf=5)
    tree.fit(x_train, y_train)
    y_pred = tree.predict(x_test)
    tree_accu = accuracy_score(y_test, y_pred)
    tree_acuracy.append(tree_accu)

    #---------------Naive Bayes-------------

    gnb = GaussianNB()
    gnb.fit(x_train,y_train)
    gnb_predic = gnb.predict(x_test)
    gnb_accu = accuracy_score(y_test,gnb_predic)
    nb_acuracy.append(gnb_accu)

    #--------------KNN - All variations-----

    knn1 = KNeighborsClassifier(n_neighbors=1)
    knn1.fit(x_train,y_train)
    knn1_predic = knn1.predict(x_test)
    knn1_accu = accuracy_score(y_test,knn1_predic)
    knn_1_acuracy.append(knn1_accu)

    knn3 = KNeighborsClassifier(n_neighbors=3)
    knn3.fit(x_train,y_train)
    knn3_predic = knn3.predict(x_test)
    knn3_accu = accuracy_score(y_test,knn3_predic)
    knn_3_acuracy.append(knn3_accu)

    knn5 = KNeighborsClassifier(n_neighbors=5)
    knn5.fit(x_train,y_train)
    knn5_predic = knn5.predict(x_test)
    knn5_accu = accuracy_score(y_test,knn5_predic)
    knn_5_acuracy.append(knn5_accu)

    #--------------WKNN - All variations-----
    wknn1 = KNeighborsClassifier(n_neighbors=1,weights='distance')
    wknn1.fit(x_train, y_train)
    wknn1_predic = wknn1.predict(x_test)
    wknn1_accu = accuracy_score(y_test, wknn1_predic)
    wknn_1_acuracy.append(wknn1_accu)

    wknn3 = KNeighborsClassifier(n_neighbors=3,weights='distance')
    wknn3.fit(x_train, y_train)
    wknn3_predic = wknn3.predict(x_test)
    wknn3_accu = accuracy_score(y_test, wknn3_predic)
    wknn_3_acuracy.append(wknn3_accu)

    wknn5 = KNeighborsClassifier(n_neighbors=5,weights='distance')
    wknn5.fit(x_train, y_train)
    wknn5_predic = wknn5.predict(x_test)
    wknn5_accu = accuracy_score(y_test, wknn5_predic)
    wknn_5_acuracy.append(wknn5_accu)

    #----------------SVM Linear--------------

    svm_linear = svm.SVC(C=1, kernel='linear')
    svm_linear.fit(x_train, y_train)
    svm_predic = svm_linear.predict(x_test)
    svml_accu = accuracy_score(y_test,svm_predic)
    svm_linear_acuracy.append(svml_accu)


    # ----------------SVM RBF--------------

    svm_rbf = svm.SVC(C=1, kernel='rbf')
    svm_rbf.fit(x_train, y_train)
    svm_predic = svm_rbf.predict(x_test)

    svmr_accu = accuracy_score(y_test, svm_predic)
    svm_rbf_acuracy.append(svmr_accu)

## Dividindo os dados em conjunto de Treino, (Validação) e Teste
### Primeiro vamos dividir os dados das classes

In [0]:
#---------Split Class column from features-------------
n_start = 0
n_final = 3
key ="clas"
X = data.iloc[:,n_start:n_final]
Y = data[key]

### Método 70 - 30

In [53]:
from sklearn.model_selection import train_test_split

#print(Y)
#----------Split Training from Test
x_train, x_test,y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)
call_classifiers(x_train, y_train, x_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


### Mérodo K-Fold

In [0]:
from sklearn.model_selection import KFold
#---------Split Class column from features-------------


kf = KFold(n_splits=10, shuffle=False)
for train_index, test_index in kf.split(X):
  x_train, x_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
  call_classifiers(x_train, y_train, x_test, y_test)

## Analisando os Resultados

### A métrica principal utilizada para avaliar classificadores é a acurácia, porém existem outras que podem ser utilizadas.

Acurácia Sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

Metricas Sklearn: https://scikit-learn.org/stable/modules/model_evaluation.html

In [54]:
def scores(acuracy_list):
    #scaler = StandardScaler()
    series = pd.Series(acuracy_list)
    media = series.mean()
    dp = series.std()
    mediana = series.median()
    return media,dp,mediana

#---------Função Fuleca só pra gerar o Log-----
def showStatistics(acuracy_list):
    mean, dp, median = scores(acuracy_list[0])
    print("###-------Algoritm:-----###",acuracy_list[1],"\n")
    print("#---Mean: ", mean,"\n")
    print("#---DP:", dp,"\n")
    print("#---Median:", median,"\n")

List_ofLists =[(tree_acuracy,"Tree"),(nb_acuracy,"Naive bayes"),(knn_1_acuracy,"KNN -1"),(knn_3_acuracy,"KNN -3"),
               (knn_5_acuracy, "KNN -5"),(wknn_1_acuracy,"WKNN-1"),(wknn_3_acuracy,"WKNN-3"),(wknn_5_acuracy,"WKNN-5"),
               (svm_linear_acuracy,"SVM-Linear"),(svm_rbf_acuracy,"SVM-RBF")]
for acuracy_list in List_ofLists:
    #print(acuracy_list[0])
    showStatistics(acuracy_list)

[1.0, 1.0, 1.0, 1.0, 0.8666666666666667, 0.8666666666666667, 0.8, 0.9333333333333333, 0.9333333333333333, 0.7333333333333333, 0.9285714285714286, 1.0]
###-------Algoritm:-----### Tree 

#---Mean:  0.9218253968253968 

#---DP: 0.08909768561110172 

#---Median: 0.9333333333333333 

[0.9333333333333333, 1.0, 1.0, 1.0, 0.7333333333333333, 1.0, 0.7333333333333333, 0.8666666666666667, 0.7333333333333333, 0.7333333333333333, 0.7857142857142857, 0.9333333333333333]
###-------Algoritm:-----### Naive bayes 

#---Mean:  0.871031746031746 

#---DP: 0.11963419702599834 

#---Median: 0.9 

[0.8666666666666667, 1.0, 1.0, 0.9333333333333333, 0.7333333333333333, 0.8, 0.8666666666666667, 0.8666666666666667, 0.8666666666666667, 0.8666666666666667, 0.7857142857142857, 0.8666666666666667]
###-------Algoritm:-----### KNN -1 

#---Mean:  0.8710317460317462 

#---DP: 0.07894005964811089 

#---Median: 0.8666666666666667 

[0.9555555555555556, 1.0, 1.0, 1.0, 0.8666666666666667, 0.7333333333333333, 0.86666666666